# A deeper look at SVM architectures

In the last experiment that we ran, where we used a LinearSVM without changing any of the deafult parameters or feaure engineering, we achieved an accuracy of 63%. It also gave us convergence warnings for the linear kernel.
In this notebook we will iterate over the SVM design and try different approaches to the problem using this classifier.
Let's import what we need.

In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from exp4_data_feature_extraction import get_balanced_dataset
from scripts import training_helpers

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

In [3]:
reviews_set, fake_reviews, genuine_reviews, unused_genuine_reviews = get_balanced_dataset()

We wish to look further into how an SVC peforms on our data, by tweaking the kernel and parameters.
Let's run the classic SVC without changing any parameters first (the default kernel is 'rbf'.)

In [ ]:
from sklearn.svm import SVC

In [4]:
all_reviews = reviews_set[:20000]

In [ ]:
cv = CountVectorizer(stop_words='english', ngram_range=(0, 2))
classifier = SVC(random_state=42) # Starting seed



In [8]:
X = [x.review_content for x in all_reviews]
y = [x.label for x in all_reviews]

In [ ]:
model = Pipeline([ ('cv', cv), ('classifier', classifier) ])
training_helpers.get_accuracy(model, X, y, 5)

SVM with a linear kernel is actually supposed to be well suited to text classification. We should however see better results if we preprocess our text to lemmatize and remove stopwords. Since bag of words is our main feature here, this should hopefully be influential. In this case we are removing all of the stopwords, which may be a bad idea. We can't know for sure unless we experiment.

In [ ]:
from exp2_feature_extraction import find_words, preprocess_words
def preprocess(review_content): # Not adding bigrams yet
  return " ".join(preprocess_words(find_words(review_content), bigrams=False))

In [ ]:
review_content = all_reviews[0].review_content
print(review_content)
preprocess(review_content)

In [5]:
cv = CountVectorizer()
classifier = LinearSVC(random_state=42) # Starting seed

In [ ]:
model = Pipeline([ ('cv', cv), ('classifier', classifier) ])

Without any processing, we have the following accuracy:

In [ ]:
We cannot find the AUC of LinearSVM because it does not give us a probability, 

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

def run_cross_validate(model, X, y, cv=5):
  skfSplitter = StratifiedKFold(n_splits=cv)
  clf = CalibratedClassifierCV(model)

  scores = []
  auc = []
  for train_indices, test_indices in skfSplitter.split(X, y):
    training_X = [X[x] for x in train_indices]
    training_y = [y[x] for x in train_indices]
    test_X = [X[x] for x in test_indices]
    test_y = [y[x] for x in test_indices]
  
    clf.fit(training_X, training_y)
    probabilities = clf.predict_proba(test_X)
    
    predicted = [0 if x[0] > x[1] else 1 for x in probabilities]
    true_probabilities = [probabilities[i][(1 if test_y[i] else 0)] for i in range(0, len(test_y))]
    
    scores.append(accuracy_score(test_y, predicted))
    auc.append(roc_auc_score(test_y, true_probabilities))
        
  return {
    "scores": scores,
    "auc": auc,
    "mean": np.mean(scores),
    "variance": np.var(scores)
  }

In [ ]:
cVec = CountVectorizer()
predictors = cVec.fit_transform(X[:5000]).toarray()

run_cross_validate(classifier, predictors, y[:5000])

/home/stefan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/stefan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [35]:
np.mean(cross_val_score(classifier, predictors, y[:1000], cv=5))

0.5839656491412285

In [ ]:
X_lemmatized = [preprocess(x.review_content) for x in all_reviews]

In [ ]:
X_set = [x.review_content for x in all_reviews[:100]]

cv = CountVectorizer()
training_X2 = cv.fit_transform(X_set)
training_X2.shape

In [ ]:
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
roc_auc_score(y_true, y_scores)

In [ ]:
run_cross_validate(model, X_lemmatized, y)

This shows only a slightly better result. Perhaps the different versions of words people use are actually important, and perhaps stopwords are important here too. At least more important than other tasks, for example identifying sentiment or topic. Let's try lemmatizing, but without removing stopwords.

In [ ]:
def preprocess(review_content): # Not adding bigrams yet
  return " ".join(preprocess_words(find_words(review_content), bigrams=False, stopwords=[]))

In [ ]:
review_content = all_reviews[0].review_content
print(review_content)
preprocess(review_content)

In [ ]:
training_helpers.get_accuracy(model, [preprocess(x.review_content) for x in all_reviews], y, 5)